In [1]:
import os
import pickle
import time

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import certifi
import ssl

# OAuth 2.0 credentials
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = r"D:\OneDrive\OneDrive - Rowad Modern Engineering\x004 Data Science\03.rme.db\00.repo\rme.db\09.api.projects\01.youtube\02.upload.with.api\client_secret_253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com.json"

def get_authenticated_service():
    creds = None
    # Check for existing credentials
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If no valid credentials, perform the OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Set the SSL context for InstalledAppFlow
            ssl_context = ssl.create_default_context(cafile=certifi.where())
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0, ssl_context=ssl_context)
        # Save credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('youtube', 'v3', credentials=creds)

def upload_to_youtube(video_path, title, description, tags=[], privacyStatus='unlisted'):
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": tags,
                "madeForKids": False
            },
            "status": {
                "privacyStatus": privacyStatus
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )
    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(" " * 80, end="\r")  # Clear the line
                print(f"Uploaded {int(status.progress() * 100)}%", end="\r")
                time.sleep(0.1)
        except Exception as e:
            print(f"An error occurred during upload: {e}")
            return
    print(f"Video uploaded successfully! Video ID: {response['id']}")

def upload_videos_from_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            title = filename[:-4]
            description = f"Uploaded from folder: {folder_path}"
            upload_to_youtube(video_path, title, description)

# Get the folder path
folder_path = input("Enter the folder path containing the videos: ")

# Upload videos from the folder
upload_videos_from_folder(folder_path)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57019%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=PnKkYJhOvszA5me1lRDYy0yxy7fauI&ssl_context=%3Cssl.SSLContext+object+at+0x0000012FC7302450%3E&access_type=offline
An error occurred during upload: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)
Video uploaded successfully! Video ID: P_NRFyvgrW0
